<a href="https://colab.research.google.com/github/vohoaidanh/AIGCDetectBenchmark/blob/main/colab/intrinsic_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Make sure to set the runtime to GPU: Runtime -> Change runtime type -> T4 GPU

You can upload your own images, then change the relevant code cells to load it and send it through the model.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# install the intrinsic decomposition repo from github
!pip install https://github.com/compphoto/Intrinsic/archive/main.zip

     / 29.2 MB 38.8 MB/s 0:00:01
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/CCareaga/MiDaS (to revision master) to /tmp/pip-install-guxwebpg/altered-midas_ba6b9ec694b94e1685dba6cf03a4cdfc
  Running command git clone --filter=blob:none --quiet https://github.com/CCareaga/MiDaS /tmp/pip-install-guxwebpg/altered-midas_ba6b9ec694b94e1685dba6cf03a4cdfc
  Resolved https://github.com/CCareaga/MiDaS to commit 043f3b8d9f6d2f7e740a8ac45e0745c0b5e2fd03
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/CCareaga/chrislib (to revision main) to /tmp/pip-install-guxwebpg/chrislib_e55c9198b79f4abb9d5bb6ab9c029bc2
  Running command git clone --filter=blob:none --quiet https://github.com/CCareaga/chrislib /tmp/pip-install-guxwebpg/chrislib_e55c9198b79f4abb9d5bb6ab9c029bc2
  Resolved https://github.com/CCareaga/chrislib to commit 6834448930593ec6f7403f411bd42beaf3afb27b
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2

In [3]:
import torch

# import some helper functions from chrislib (will be installed by the intrinsic repo)
from chrislib.general import show, view, uninvert
from chrislib.data_util import load_image

# import model loading and running the pipeline
from intrinsic.pipeline import run_pipeline
from intrinsic.model_util import load_models

In [4]:
# download the pretrained weights and return the model (may take a bit to download weights)
intrinsic_model = load_models('paper_weights')

Downloading: "https://github.com/compphoto/Intrinsic/releases/download/v1.0/final_weights.pt" to /root/.cache/torch/hub/checkpoints/final_weights.pt
100%|██████████| 485M/485M [00:06<00:00, 83.0MB/s]
Downloading: "https://github.com/facebookresearch/WSL-Images/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://download.pytorch.org/models/ig_resnext101_32x8-c38310e5.pth" to /root/.cache/torch/hub/checkpoints/ig_resnext101_32x8-c38310e5.pth
100%|██████████| 340M/340M [00:01<00:00, 239MB/s]
/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., 

In [5]:
# load an example image from the github repo
torch.hub.download_url_to_file('https://raw.githubusercontent.com/compphoto/Intrinsic/main/figures/avocado.png', 'avo.png')

100%|██████████| 2.85M/2.85M [00:00<00:00, 59.3MB/s]


In [164]:
images[187+1823]

'/content/real/n13133613/ILSVRC2012_val_00012125.JPEG'

In [174]:
# load the image to run through the pipeline
img = load_image(images[185+1824])

In [176]:
img.shape

(461, 614, 4)

In [175]:
# run the image through the pipeline (use R0 resizing dicussed in the paper)
result = run_pipeline(
    intrinsic_model,
    img,
    resize_conf=0.0,
    maintain_size=True,
    linear=False,
    device='cuda'
)

RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[1, 4, 480, 640] to have 3 channels, but got 4 channels instead

In [8]:
# convert the inverse shading to regular shading for visualization
shd = uninvert(result['inv_shading'])
alb = result['albedo']

In [12]:
alb.shape

(1152, 1504, 3)

In [ ]:
# show the result (gamma corrects the linear intrinsic components and scales to [0-1])
show([img, view(shd), view(alb)], size=(20, 7))

In [44]:
import os

def get_image_list(data_path):
  # Get all images in dataset (return a list of path of each image)
  items = os.listdir(data_path) # Each item contains 40 images
  image_list = []
  for item in items:
    f = os.path.join(data_path, item)
    imgs = os.listdir(f)
    imgs_1 = [os.path.join(f,i) for i in imgs]
    image_list.extend(imgs_1)
  return image_list


In [90]:
from PIL import Image
import os
import numpy as np
def save_image(image_, folder_path, filename):
    """
    Save an image to a specified folder.
    """
    image = (view(image_) * 255.0).astype(np.uint8)

    if len(image.shape) == 3 and image.shape[2] == 1:
        image = np.squeeze(image, axis=2)

    if image.dtype != np.uint8:
        image = image.astype(np.uint8)

    image = Image.fromarray(image)

    # Create the folder if it doesn't exist
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Construct the full file path
    file_path = os.path.join(folder_path, filename)

    # Save the image
    image.save(file_path)


In [15]:
!cp /content/drive/MyDrive/DATASETS/DiffusionForensics/imagenet/train/real.tar.gz -d /content

In [16]:
!gunzip /content/real.tar.gz

In [ ]:
!tar -xvf /content/real.tar

In [23]:
'data\{0}'.format('n001')

'data\\n001'

# Run process all images in the list

In [45]:
data_path = '/content/real'
dest_root = '/content/drive/MyDrive/DATASETS/DiffusionForensics/intrinsic/imagenet/train/0_real'

images = get_image_list(data_path)
from pathlib import Path


In [ ]:
from tqdm import tqdm
print('Total: {0} images'.format(len(images)))
albedo = None
inv_shd = None
shading = None

for image in tqdm(images[185+1820+78:]):
    img = load_image(image)

    if (img.shape[0]) < 200: continue
    if (img.shape[1]) < 200: continue


    if img.shape[-1] != 3:
      print(image)
      continue
    try:
      result = run_pipeline(
        intrinsic_model,
        img,
        resize_conf=0.0,
        maintain_size=True,
        linear=False,
        device='cuda'
      )
    except ValueError as err:
      print(err," : ", image)
      continue

    albedo = result['albedo']
    inv_shd = result['inv_shading']

    # compute shading from inverse shading
    shading = uninvert(inv_shd)
    # File name
    file_name = Path(image).stem

    save_image(albedo, dest_root,file_name + '_albedo_' + '.png')
    save_image(shading, dest_root,file_name + '_shading_' + '.png')
    save_image(img, dest_root,file_name + '_origin_' + '.png')





Total: 40000 images


  0%|          | 1/37917 [00:00<8:52:00,  1.19it/s]

/content/real/n03063689/ILSVRC2012_val_00002078.JPEG


  0%|          | 11/37917 [00:12<14:18:36,  1.36s/it]

/content/real/n03063689/ILSVRC2012_val_00022585.JPEG


...

  8%|▊         | 2950/37917 [1:00:16<10:26:33,  1.08s/it]

/content/real/n02999410/ILSVRC2012_val_00017849.JPEG


  8%|▊         | 2995/37917 [1:01:01<9:43:25,  1.00s/it] 

/content/real/n01494475/ILSVRC2012_val_00039588.JPEG


  8%|▊         | 3099/37917 [1:02:57<11:10:47,  1.16s/it]

'/content/real/n13133613/ILSVRC2012_val_00009549.JPEG'